# Train test split WPCC with propeller model
<cite id="hzn48"><a href="#zotero|7505983/U2L55C9Q">(Luo et al., 2016)</a></cite>

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression, create_full_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.models.vmm import VMM
from src.parameters import df_parameters
from src.models.propeller import fit, predict, predictor
from wPCC_pipeline.pipelines.kvlcc2.nodes import fit_propeller_characteristics

In [ ]:
ship="wpcc"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)
ship_data = catalog.load(f"{ship}.ship_data")
data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")
data['rev'] = data[[r'Prop/PS/Rpm',r'Prop/SB/Rpm']].mean(axis=1)
added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
open_water_characteristics = catalog.load(f"{ship}.open_water_characteristics")
propeller_coefficients = catalog.load(f"{ship}.propeller_coefficients")

In [ ]:
kt = open_water_characteristics['Kt']
kq = open_water_characteristics['Kq']
J = open_water_characteristics.index

#propeller_coefficients = fit_propeller_characteristics(open_water_characteristics=open_water_characteristics)
#propeller_coefficients = pd.Series(propeller_coefficients)
coefficients = np.flipud(list(propeller_coefficients.values()))
#coefficients = propeller_coefficients.values
Kt = np.polyval(p=coefficients, x=J)

open_water_characteristics['10*kq'] = 10*open_water_characteristics['Kq']
open_water_characteristics['eta0'] = kt/(kq*2*np.pi)*J
fig,ax=plt.subplots()
open_water_characteristics.plot(y=['Kt','10*kq','eta0'], ax=ax)

ax.plot(J,Kt,'k--',)

In [ ]:
data.head()

In [ ]:
ids = data['id'].unique()

In [ ]:
np.random.seed(11)
ids_test = np.random.choice(ids, size=int(np.ceil(len(ids)*0.4)), replace=False)
ids_train = list(set(ids) - set(ids_test))

mask = data['id'].isin(ids_train)
df_train = data.loc[mask].copy()

In [ ]:
dataframes_test = {}
for id in ids_test:
    dataframes_test[id] = catalog.load(f'{ship}.updated.{id}.data_ek_smooth')

In [ ]:
track_plots(dataframes_test, lpp=ship_data['L'], beam=ship_data['B']);

In [ ]:
plot(dataframes=dataframes_test, keys=['delta','psi','thrust'], ncols=1);

In [ ]:
df_train = df_train[["x0","y0","z0","u",'v','r','u1d','v1d','r1d','delta','thrust','rev']].copy()
data_with_force = predict_force(data=df_train, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)

In [ ]:
vmm.X_eq

In [ ]:
vmm.Y_eq

In [ ]:
vmm.N_eq

In [ ]:
regression, parameters = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
model = create_model_from_motion_regression(regression=regression)

## Thrust model

In [ ]:
#from src.models.propeller import preprocess, lambda_w_p, lambda_thrust_simple
#from src.substitute_dynamic_symbols import run
#import statsmodels.api as sm
#
#def features(df, ship_data: dict, add_constant=False):
#
#    if isinstance(df, pd.DataFrame):
#        X = pd.DataFrame(index=df.index.copy())
#    else:
#        X = pd.DataFrame(index=[0])
#
#    twin = ship_data['TWIN'] == 1
#    X["delta"] = 0
#    
#    
#    X["delta**2"] = df["delta"] ** 2
#    v_p = df["v"] + df["r"] * ship_data["x_p"]
#    X["v_p"] = 0
#    X["v_p**2"] = v_p ** 2
#
#    if add_constant:
#        X = sm.tools.add_constant(X, has_constant="add")
#
#    return X
#
#
#def fit(
#    data: pd.DataFrame,
#    ship_data: dict,
#    propeller_coefficients: dict,
#    add_constant=False,
#):
#
#    data = preprocess(
#        data, ship_data=ship_data, propeller_coefficients=propeller_coefficients
#    )
#    
#    if ship_data['TWIN'] == 1:
#        data = data.copy()
#        data['thrust']/=2
#    
#    data["w_p"] = run(lambda_w_p, inputs=data, **ship_data, **propeller_coefficients)
#
#    df_pos = data.copy()
#    X = features(df_pos, ship_data=ship_data, add_constant=add_constant)
#    y = df_pos["w_p"] - ship_data["w_p0"]
#    linear_regression_pos = sm.OLS(y, X, hasconst=add_constant)
#    model_pos = linear_regression_pos.fit()
#
#    df_neg = data.copy()
#
#    X = features(df_neg, ship_data=ship_data, add_constant=add_constant)
#    y = df_neg["w_p"] - ship_data["w_p0"]
#    linear_regression_neg = sm.OLS(y, X, hasconst=add_constant)
#    model_neg = linear_regression_neg.fit()
#
#    return model_pos, model_neg

In [ ]:
add_constant = True
model_pos, model_neg = fit(data=df_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 
data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=data_with_force, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data)

regression_predicted_thrust, parameters = fit_motions(data=data_predicted_thrust, added_masses=added_masses, 
                                                      ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
fig,ax=plt.subplots()
df_train.plot(y='thrust', ax=ax)
data_predicted_thrust.plot(y='thrust', ax=ax)

In [ ]:
model_predicted_thrust = create_full_model_from_motion_regression(regression=regression_predicted_thrust, 
                                                                  model_pos=model_pos, 
                                                                  model_neg=model_neg, 
                                                                  propeller_coefficients=propeller_coefficients)

In [ ]:
model_pos.summary()

In [ ]:
for id in ids_test:
    
    df_test = catalog.load(f'{ship}.updated.{id}.data_ek_smooth')
    df_test['rev'] = df_test[[r'Prop/PS/Rpm',r'Prop/SB/Rpm']].mean(axis=1)
    
    df_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=df_test, propeller_coefficients=propeller_coefficients, 
                                    ship_data=ship_data)
        
    result = model.simulate(df_=df_test)
    result_predicted_thrust = model_predicted_thrust.simulate(df_=df_test.drop(columns=['thrust']))
    
    df_result = result.result.copy()
    df_result_predicted_thrust = result_predicted_thrust.result.copy()
    df_result_predicted_thrust['thrust'] = predict(model_pos=model_pos, 
                                                   model_neg=model_neg, 
                                                   data=df_result_predicted_thrust, 
                                                   propeller_coefficients=propeller_coefficients, 
                                                   ship_data=ship_data)['thrust']
    
    for df in [df_test, df_result, df_result_predicted_thrust]:
        df['psi_deg'] = np.rad2deg(df['psi'])
        df['delta_deg'] = np.rad2deg(df['delta'])
    
    dataframes = {
        'Experiment' : df_test,
        'Hull + Rudder prediction' : df_result,
        'Full prediction' : df_result_predicted_thrust,
        
    }
    
    styles = {
        'Experiment' : {'style':'r--'},
        'Hull + Rudder prediction' : {'style':'b:'},
        'Full prediction' : {'style':'g-'},
    }
    
    track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);
    plot(dataframes=dataframes, keys=['delta_deg','psi_deg','thrust','y0','u','v','r'], ncols=1, styles=styles);

In [ ]:
u0 = df_test['u'].iloc[0]
rev = df_test['rev'].iloc[0]
result2 = model_predicted_thrust.zigzag(u0=u0, rev=rev, angle=20, heading_dev=10)
result2.include_accelerations=False
df_result2 = result2.result.copy()
for df in [df_result2]:
    df['psi_deg'] = np.rad2deg(df['psi'])
    df['delta_deg'] = np.rad2deg(df['delta'])

In [ ]:
df_result2['thrust'] = predict(model_pos=model_pos, model_neg=model_neg, data=df_result2, propeller_coefficients=propeller_coefficients, ship_data=ship_data)['thrust']

In [ ]:
track_plots({'sim':result2.result,'test':df_test}, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot({'sim':result2.result,'test':df_test}, keys=['delta','psi','thrust','y0','u','v','r'], ncols=1, time_window=[0,10]);

In [ ]:
u0 = data['u'].iloc[0]
rev = data['rev'].iloc[0]
result2 = model_predicted_thrust.turning_circle(u0=u0, rev=rev, angle=35)
result2.include_accelerations=False
df_result2 = result2.result.copy()

In [ ]:
df_result2['thrust'] = predict(model_pos=model_pos, model_neg=model_neg, data=df_result2, propeller_coefficients=propeller_coefficients, ship_data=ship_data)['thrust']

In [ ]:
track_plots({'sim':result2.result}, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot({'sim':df_result2}, keys=['delta','psi','thrust','y0','u','v','r'], ncols=1,);

In [ ]:
5.0/41.2